You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

Your task is to first identify the three best-performing industries based on the number of new unicorns created in 2019, 2020, and 2021 combined.

From those industries (1), you will need to find the number of unicorns within these industries (2), the year that they became a unicorn (3), and their average valuation, converted to billions of dollars and rounded to two decimal places (4).

With the above information you can then finish your query to return a table containing industry, year, num_unicorns, and average_valuation_billions. For readability, the firm have asked you to sort your results by year and number of unicorns, both in descending order.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# Identifying the three industries with highest number of new unicorns in the years (2019-2021)

In [10]:

WITH industry_unicorn_counts AS (
    SELECT 
        i.industry,
        COUNT(DISTINCT i.company_id) AS total_unicorns
    FROM industries i
    JOIN dates d ON i.company_id = d.company_id
    WHERE EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
    GROUP BY i.industry
    ORDER BY total_unicorns DESC
    LIMIT 3
)
SELECT 
    industry,
    total_unicorns AS "Total New Unicorns (2019-2021)"
FROM industry_unicorn_counts;

,industry,Total New Unicorns (2019-2021)
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


# Year-by-year breakdown of unicorn creation in top industries

In [11]:
WITH top_industries AS (
    SELECT 
        i.industry
    FROM industries i
    JOIN dates d ON i.company_id = d.company_id
    WHERE EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
    GROUP BY i.industry
    ORDER BY COUNT(DISTINCT i.company_id) DESC
    LIMIT 3
),
yearly_growth AS (
    SELECT 
        i.industry,
        EXTRACT(YEAR FROM d.date_joined) AS year,
        COUNT(DISTINCT i.company_id) AS unicorns_created
    FROM industries i
    JOIN dates d ON i.company_id = d.company_id
    WHERE i.industry IN (SELECT industry FROM top_industries)
      AND EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
    GROUP BY i.industry, EXTRACT(YEAR FROM d.date_joined)
    ORDER BY year, unicorns_created DESC
)
SELECT 
    industry,
    year,
    unicorns_created,
    ROUND(100.0 * unicorns_created / SUM(unicorns_created) OVER (PARTITION BY industry), 2) AS "Percentage of 3-Year Total"
FROM yearly_growth
ORDER BY year DESC, unicorns_created DESC;

,industry,year,unicorns_created,Percentage of 3-Year Total
0,Fintech,2021,138,79.77
1,Internet software & services,2021,119,78.29
2,E-commerce & direct-to-consumer,2021,47,62.67
3,Internet software & services,2020,20,13.16
4,E-commerce & direct-to-consumer,2020,16,21.33
5,Fintech,2020,15,8.67
6,Fintech,2019,20,11.56
7,Internet software & services,2019,13,8.55
8,E-commerce & direct-to-consumer,2019,12,16.00


# Valuation analysis for top industries by year

In [12]:
WITH top_industries AS (
    SELECT 
        i.industry
    FROM industries i
    JOIN dates d ON i.company_id = d.company_id
    WHERE EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
    GROUP BY i.industry
    ORDER BY COUNT(DISTINCT i.company_id) DESC
    LIMIT 3
),
valuation_stats AS (
    SELECT 
        i.industry,
        EXTRACT(YEAR FROM d.date_joined) AS year,
        COUNT(DISTINCT i.company_id) AS num_unicorns,
        ROUND(AVG(f.valuation/1000000000.0), 2) AS average_valuation_billions,
        ROUND(MIN(f.valuation/1000000000.0), 2) AS min_valuation_billions,
        ROUND(MAX(f.valuation/1000000000.0), 2) AS max_valuation_billions,
        ROUND(STDDEV(f.valuation/1000000000.0), 2) AS stddev_valuation_billions
    FROM industries i
    JOIN dates d ON i.company_id = d.company_id
    JOIN funding f ON i.company_id = f.company_id
    WHERE i.industry IN (SELECT industry FROM top_industries)
      AND EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
    GROUP BY i.industry, EXTRACT(YEAR FROM d.date_joined)
)
SELECT 
    industry,
    year,
    num_unicorns,
    average_valuation_billions,
    min_valuation_billions,
    max_valuation_billions,
    stddev_valuation_billions
FROM valuation_stats
ORDER BY year DESC, num_unicorns DESC;

,industry,year,num_unicorns,average_valuation_billions,min_valuation_billions,max_valuation_billions,stddev_valuation_billions
0,Fintech,2021,138,2.75,1.0,32.0,3.41
1,Internet software & services,2021,119,2.15,1.0,12.0,1.55
2,E-commerce & direct-to-consumer,2021,47,2.47,1.0,13.0,2.47
3,Internet software & services,2020,20,4.35,1.0,10.0,3.01
4,E-commerce & direct-to-consumer,2020,16,4.00,1.0,15.0,4.27
5,Fintech,2020,15,4.33,1.0,8.0,2.41
6,Fintech,2019,20,6.80,1.0,40.0,9.87
7,Internet software & services,2019,13,4.23,1.0,13.0,3.32
8,E-commerce & direct-to-consumer,2019,12,2.58,1.0,5.0,1.56


# Industry performance metrics by year

In [13]:
WITH top_industries AS (
    SELECT 
        i.industry
    FROM industries i
    JOIN dates d ON i.company_id = d.company_id
    WHERE EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
    GROUP BY i.industry
    ORDER BY COUNT(DISTINCT i.company_id) DESC
    LIMIT 3
)

SELECT 
    i.industry,
    EXTRACT(YEAR FROM d.date_joined)::INTEGER AS year,
    COUNT(DISTINCT i.company_id) AS num_unicorns,
    ROUND(AVG(f.valuation/1000000000.0), 2) AS average_valuation_billions
FROM industries i
JOIN dates d ON i.company_id = d.company_id
JOIN funding f ON i.company_id = f.company_id
WHERE i.industry IN (SELECT industry FROM top_industries)
  AND EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
GROUP BY i.industry, EXTRACT(YEAR FROM d.date_joined)
ORDER BY year DESC, num_unicorns DESC;

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58
